<a href="https://colab.research.google.com/github/gibranfp/CursoDatosMasivosI/blob/main/notebooks/4c_elementos_distintos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Algoritmo de Flajolet-Martin
En esta libreta programaremos el algoritmo Flajolet-Martin  para estimar el número de elementos distintos en un flujo de datos.

La idea detrás de este algoritmo es que entre más elementos diferentes haya en el flujo de datos, más valores _hash_ diferentes veremos y es más probable que alguno de estos valores tenga una representación binaria que termine con un mayor número de ceros consecutivos.

En particular, sea $tam(x)$ el número de 0s al final de la cadena correspondiente al valor _hash_ del elemento $x$, se inicializa un arreglo de bits de tamaño $L$ con ceros y se pone a 1 el bit de la posición $tam(x)$ para cada $x$ del flujo. Sea $r$ la primera posición del arreglo de bits cuyo valor es cero, un estimador del número de elementos en el flujo de datos es $\frac{2^{r}}{\phi}$, donde $\phi \approx 0.77351$ es un factor de corrección.

In [1]:
import numpy as np
np.random.seed(2021) # para reproducibilidad

class ConteoProbabilista:  
  def __init__(self, n_cubetas, primo, n_bits=64):
    self.primo = primo  
    self.n_cubetas = n_cubetas
    self.a = np.random.randint(1, self.primo - 1)
    self.b = np.random.randint(0, self.primo - 1)
    self.bitmap = np.zeros(n_bits, dtype=np.bool)

  def __call__(self, x):
    hv = ((self.a * x + self.b) % self.primo) % self.n_cubetas
    i = bin(hv)[2:][::-1].find('1')
    self.bitmap[i] = 1

  def cardinalidad(self):
    r = np.argwhere(self.bitmap == 0)[0]
    return (2**r) / 0.77351

Definimos una clase que realiza varias estimaciones, las divide en grupos pequeños, obtiene la mediana de las estimaciones de cada grupo y toma el promedio de las medianas como estimación final.

In [2]:
class EstimadorElementosDistintos:
  def __init__(self, n_cubetas, n_grupos, n_funciones, primo, n_bits):
    self.n_grupos = n_grupos
    self.n_funciones = n_funciones
    self.estimadores = []
    for i in range(self.n_grupos):
      func = []
      for j in range(self.n_funciones):
        func.append(ConteoProbabilista(n_cubetas, primo, n_bits))
      self.estimadores.append(func)
    self.conteos = np.zeros((self.n_grupos, self.n_funciones))

  def __call__(self, x):
    for i in range(self.n_grupos):
      for j in range(self.n_funciones):  
        self.estimadores[i][j](x)
        self.conteos[i, j] = self.estimadores[i][j].cardinalidad()
      
  def cardinalidad(self):
    return np.mean(np.median(self.conteos, axis=1))

Generamos números aleatorios.

In [3]:
import numpy as np

X = np.random.randint(0,100000, size=1000000)
print("Hay {0} elementos distintos".format(np.unique(X).size))

Hay 99995 elementos distintos


Instanciamos nuestra clase y estimamos elementos distintos.

In [4]:
est = EstimadorElementosDistintos(10000000, 5, 10, 4294967291, 64)

for i,x in enumerate(X):
  est(x)
  
print(u'Real = {0} Estimación = {1} '.format(np.unique(X[:i+1]).size, 
                                             est.cardinalidad()))

Real = 99995 Estimación = 84725.47219816162 
